In [1]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [2]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [3]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [4]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [ ]:
# TX_CURR_METADATA
tx_curr_metadata = api.get('dataElementOperands', params={'filter':'dataElement.id:eq:p7EP9qMVwqg', 'paging':'false', 'fields':'id,name'}).json()['dataElementOperands']
for metadata in tx_curr_metadata:
    metadata_key = metadata['name'].split('MER_TX_CURR (N, TA, Age/Sex/HIVStatus)_Monthly')[1]
    metadata_key = metadata_key.replace('MER25_', '')
    metadata_key = metadata_key.replace('MER_', '')
    metadata_key = metadata_key.replace('Positive', '')
    metadata_key = metadata_key.replace(', ', '_')
    metadata['metadata_key'] = metadata_key

# TX_NEW_METADATA
tx_new_metadata = api.get('dataElementOperands', params={'filter':'dataElement.id:eq:xS1Lke6V3Mi', 'paging':'false', 'fields':'id,name'}).json()['dataElementOperands']
for metadata in tx_new_metadata:
    metadata_key = metadata['name'].split('TX_NEW (N, TA, Age/Sex/CD4/HIVStatus)_Monthly')[1]
    metadata_key = metadata_key.replace('MER25_', '')
    metadata_key = metadata_key.replace('MER_', '')
    metadata_key = metadata_key.replace(', ', '_')
    metadata['metadata_key'] = metadata_key


# MISSED_APPOINTMENT
missed_appointment_metadata = api.get('dataElementOperands', params={'filter':'dataElement.id:eq:afMU582wyjr', 'paging':'false', 'fields':'id,name'}).json()['dataElementOperands']
for metadata in missed_appointment_metadata:
    metadata_key = metadata['name'].split('MISSED_APPOINTMENT (N, TA, Age/Sex/HIVStatus)_Monthly')[1]
    metadata_key = metadata_key.replace('MER25_', '')
    metadata_key = metadata_key.replace('MER_', '')
    metadata_key = metadata_key.replace(', ', '_')
    metadata['metadata_key'] = metadata_key

In [ ]:
# enrollments = pd.DataFrame(columns=['enrollment', 'trackedEntity', 'program', 'status', 'orgUnit', 'enrolledAt', 'patientIdentifier', 'patientAge', 'patientSex','patientName', 
#                                     'artStartDate', 'firstConsultationDate','pickupQuantity', 'lastPickupDate', 'nextPickupDate', 'lastCD4', 'viralLoadResultDate', 
#                                     'viralLoadResultValue', 'transferedOut', 'dateOfTransfer','dead', 'dateOfDeath'])

# enrollments.to_csv('ENROLLMENTS.csv', index=False)

In [ ]:
# data = pd.DataFrame(columns=['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value'])
# data.to_csv('DATA.csv', index=False)

In [5]:
# Indicator preparattion
config = FileUtil.load_logging_config()
logging_config = LoggingConfig(config)
logger = logging_config.logging_setup()

tx_curr_service = ComputeTxCurrService(logger)
tx_new_service = ComputeTxNewService(tx_curr_service, logger)

tx_curr_indicator_metadata_port = TxCurrIndicatorMetadataAdapter(api)
arv_dispense_indicator_metadata_port = ArvDispenseIndicatorMetadataAdapter(api)
tx_new_indicator_metadata_port = TxNewIndicatorMetadataAdapter(api)

tx_curr_disaggregation_service = ComputeTxCurrDisaggregationService(tx_curr_indicator_metadata_port, arv_dispense_indicator_metadata_port, logger)
tx_new_disaggretation_service = ComputeTxNewDisaggregationService(tx_new_indicator_metadata_port, logger)

tx_ml_indicator_metadata_port = TxMlIndicatorMetadataAdapter(api)
tx_ml_service = ComputeTxMlService(logger)
tx_ml_disaggregation_service = ComputeTxMlDisaggregationService(tx_ml_indicator_metadata_port)

patient_consultation_form = PatientConsultationForm(api)
consultation_port = ConsultationAdapter(patient_consultation_form)
patient_laboratoty_form = PatientLaboratoryForm(api)
laboratory_port = LaboratoryAdapter(patient_laboratoty_form)

tx_pvls_denominator_service = ComputeTxPvlsDenominatorService(logger, tx_curr_service, laboratory_port, consultation_port)
tx_pvls_numerator_service = ComputeTxPvlsNumeratorService()

tx_pvls_denominator_indicator_metadata_port = TxPvlsDenominatorIndicatorMetadataAdapter(api)
tx_pvls_denominator_pred_breast_indicator_metadata_port = TxPvlsDenominatorPregBreastIndicatorMetadataAdapter(api)
tx_pvls_denominator_disaggregation_service = ComputeTxPvlsDenominatorDisaggregationService(tx_pvls_denominator_indicator_metadata_port, tx_pvls_denominator_pred_breast_indicator_metadata_port)

tx_pvls_numerator_indicator_metadata_port = TxPvlsNumeratorIndicatorMetadataAdapter(api)
tx_pvls_numerator_pred_breast_indicator_metadata_port = TxPvlsNumeratorPregBreastIndicatorMetadataAdapter(api)
tx_pvls_numerator_disaggregation_service = ComputeTxPvlsNumeratorDisaggregationService(tx_pvls_numerator_indicator_metadata_port, tx_pvls_numerator_pred_breast_indicator_metadata_port)

pharmacy_form = PatientPharmacyForm(logger, api)
pharmacy_port = PharmacyAdapter(pharmacy_form)
tx_rtt_service = ComputeTxRttService(logger, pharmacy_port, tx_curr_service)
tx_rtt_indicator_metadata_port = TxRttIndicatorMetadataAdapter(api)
tx_rtt_itt_indicator_metadata_port = TxRttIttIndicatorMetadataAdapter(api)
tx_rtt_disaggregation_service = ComputeTxRttDisaggregationService(logger, tx_rtt_indicator_metadata_port, tx_rtt_itt_indicator_metadata_port)

In [ ]:
# Load patients enrolled
for org_unit in org_units:

    org_unit = org_unit['id']

    #get all patient enrollments
    patients_enrolled = api.get('tracker/enrollments', params={'orgUnit':org_unit, 'skipPaging':'true', 'program': CARE_AND_TREATMENT, 'fields':'{,enrollment, enrolledAt, orgUnit, trackedEntity, program, status,}'})
    patients_enrolled = patients_enrolled.json()['instances']

    logger.info(f"Processing the facility: {org_unit}, a total of {len(patients_enrolled)} enrolled patient(s)")
    
    counter = 1

    for patient_enrolled in patients_enrolled:
        patient_id = patient_enrolled['trackedEntity']

        patient_demographics = PatientDemographicForm(api)
        patient_demographics.add_demographics(patient_enrolled)

        patient_consultation= PatientConsultationForm(patient_id, org_unit, api)
        patient_consultation.add_first_consultation(patient_enrolled)

        patient_pharmacy_form =  PatientPharmacyForm(patient_id, org_unit, api)
        patient_pharmacy_form.add_last_pharmacy(patient_enrolled, end_period)

        patient_laboratory_from =  PatientLaboratoryForm(patient_id, org_unit, api)
        patient_laboratory_from.add_laboratory(patient_enrolled)

        patient_outcome_form =  PatientOutcomeForm(patient_id, org_unit, api)
        patient_outcome_form.add_final_outcome(patient_enrolled)

        logger.info(f"From {len (patients_enrolled)} patietnts enrolled, {counter} (is) are ready to be processed.")
        counter = counter + 1 

    if patients_enrolled:
        patients_enrolled = pd.json_normalize(patients_enrolled)
        
        enrollments = pd.read_csv('ENROLLMENTS.csv')
        enrollments = pd.concat([enrollments, patients_enrolled])
        enrollments.to_csv('ENROLLMENTS.csv', index=False, encoding='utf-8')

In [6]:
enrollments = pd.read_csv('ENROLLMENTS.csv')
enrollments = enrollments.to_dict(orient='records')

# tx_curr_patients = tx_curr_service.compute(enrollments, end_period)
# tx_curr_patients_disaggregation = tx_curr_disaggregation_service.compute(tx_curr_patients, end_period)

# tx_new_patients = tx_new_service.compute(tx_curr_patients, start_period, end_period)
# tx_new_patients_disaggregation = tx_new_disaggretation_service.compute(tx_new_patients, end_period)

# tx_ml_patients = tx_ml_service.compute(enrollments, start_period, end_period, ComputeTxMlService.QUARTERLY_DAYS_EXPECTED)
# tx_ml_patients_disaggregation = tx_ml_disaggregation_service.compute(tx_ml_patients, end_period)

# tx_pvls_denominator_patients = tx_pvls_denominator_service.compute(tx_curr_patients, end_period)
# tx_pvls_denominator_patients_disaggregation = tx_pvls_denominator_disaggregation_service.compute(tx_pvls_denominator_patients, end_period)

# tx_pvls_numerator_patients = tx_pvls_numerator_service.compute(tx_pvls_denominator_patients)
# tx_pvls_numerator_patients_disaggregation = tx_pvls_numerator_disaggregation_service.compute(tx_pvls_numerator_patients, end_period)

tx_rtt_patients = tx_rtt_service.compute(enrollments, start_period, end_period)
tx_rtt_patients_disaggregation = tx_rtt_disaggregation_service.compute(tx_rtt_patients, end_period)

# combination = tx_curr_patients_disaggregation + tx_new_patients_disaggregation + tx_ml_patients_disaggregation + tx_pvls_denominator_patients_disaggregation + tx_pvls_numerator_patients_disaggregation + tx_rtt_patients_disaggregation
combination = tx_rtt_patients_disaggregation

if combination:
    # extract data
    indicators = pd.json_normalize(combination)
    indicators['period'] = period
    indicators = indicators[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
    indicators = indicators.sort_values(['orgUnit', 'dataElement', 'period'])
    indicators.to_csv('RTT_DATA.csv', index=False)


In [ ]:
# tx_curr_service = ComputeTxCurrService()
# tx_curr_service.compute(patients, end_period)

# tx_new = ComputeTxNewService(tx_curr_service)
# tx_new.compute(patients,start_period,end_period)

# tx_ml = ComputeTxMlService()
# tx_ml.compute(patients, start_period, end_period, MO)

# laboratory_port = LaboratoryAdapter(patient_laboratory_from)
# tx_pvls_denominator = ComputeTxPvlsDenominatorService(tx_curr_service, laboratory_port)
# tx_pvls_denominator.compute(patients, end_period)

# tx_pvls_numerator = ComputeTxPvlsNumeratorService()
# tx_pvls_numerator.compute(patients)

In [ ]:
# data = pd.json_normalize(combination)
# data = data[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
# data.to_excel('DATA.xlsx', index=False)

In [ ]:
st = patient_events = api.get('tracker/events', params={'skipPaging':True, 'fields':'event, status, program, trackedEntity, orgUnit, occurredAt, dataValues[dataElement,value]', 'program':'grpiGkcSlNN', 'orgUnit':'O6AwtjumLhU', 'occurredAfter':'2024-05-01', 'occurredBefore':'2024-05-31', 'order':'occurredAt:asc'})

In [ ]:
st =st.json()['instances']

In [ ]:
st[0]['dataValues']

In [ ]:
hts_data = pd.DataFrame(columns=['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value'])
hts_data.to_csv('HTS_DATA.csv', index=False)

patient_event_form = PatientEventForm(api)
patient_demographic_form = PatientDemographicForm(api)
event_port = EventAdapter(patient_event_form)
patient_demographics_port = PatientDemographicsAdapter(patient_demographic_form)

compute_hts_vct = ComputeHtsVctService(event_port, patient_demographics_port)
vct_metadata_port = VctIndicatorMetadataAdapter(api)
vct_patients_disaggregation = ComputeHtsDisaggregationService(vct_metadata_port)

compute_hts_pregnant = ComputeHtsPregnantLdService(event_port, patient_demographics_port)
pregnant_metadata_port = PregnantIndicatorMetadataAdapter(api)
pregnant_patients_disaggregation = ComputeHtsDisaggregationService(pregnant_metadata_port)

compute_hts_breastfeeding = ComputeHtsBreastfeedingService(event_port, patient_demographics_port)
breastfeeding_metadata_port = BreastfeedingIndicatorMetadataAdapter(api)
breastfeeding_patients_disaggregation = ComputeHtsDisaggregationService(breastfeeding_metadata_port)

compute_hts_other_pitc = ComputeHtsOtherPitcService(event_port, patient_demographics_port)
other_pitc_metadata_port = OtherPitcIndicadorMetadataAdapter(api)
other_pitc_patients_disaggregation = ComputeHtsDisaggregationService(other_pitc_metadata_port)

compute_hts_emergence_ward = ComputeHtsEmergenceWardService(event_port, patient_demographics_port)
emergence_ward_port = EmergenceWardIndicatorMetadataAdapter(api)
emergenc_ward_patients_disaggregation = ComputeHtsDisaggregationService(emergence_ward_port)

compute_hts_inpatient = ComputeHtsInpatientService(event_port, patient_demographics_port)
inpatient_port = InpatientIndicatorMetadataAdapter(api)
inpatient_patients_disaggregation = ComputeHtsDisaggregationService(inpatient_port)

compute_hts_malnutrition = ComputeHtsMalnutritionService(event_port, patient_demographics_port)
malnutrition_port = MalnutritionIndicatorMetadataAdapter(api)
malnutrition_patients_disaggregation = ComputeHtsDisaggregationService(malnutrition_port)

compute_hts_index_case = ComputeHtsIndexCaseService(event_port)
index_case_port = IndexCaseIndicatorMetadataAdapter(api)
index_case_patients_disaggregation = ComputeHtsDisaggregationService(index_case_port)

compute_hts_pmtct_anc = ComputeHtsPmtctAncService(event_port, patient_demographics_port)
pmtct_anc_port = PmtctIndicatorMetadataAdapter(api)
pmtct_anc_patients_disaggregation = ComputeHtsDisaggregationService(pmtct_anc_port)

compute_hts_tb_clinic = ComputeHtsTbClinicService(event_port, patient_demographics_port)
tb_clinic_port = TbClinicIndicatorMetadataAdapter(api)
tb_clinic_patients_disaggregattion = ComputeHtsDisaggregationService(tb_clinic_port)

for org_unit in org_units:

    org_unit = org_unit['id']

    vct_patients = compute_hts_vct.compute(org_unit, start_period, end_period)
    vct_patients_disaggregated = vct_patients_disaggregation.compute(vct_patients, end_period)

    pregnant_patients = compute_hts_pregnant.compute(org_unit, start_period, end_period)
    pregnant_patients_disaggregated = pregnant_patients_disaggregation.compute(pregnant_patients, end_period)

    breastfeeding_patients = compute_hts_breastfeeding.compute(org_unit, start_period, end_period)
    breastfeeding_patients_disaggregated = breastfeeding_patients_disaggregation.compute(breastfeeding_patients, end_period)

    other_pitc_patients = compute_hts_other_pitc.compute(org_unit, start_period, end_period)
    other_pitc_patients_disaggregated = other_pitc_patients_disaggregation.compute(other_pitc_patients, end_period)

    emergence_ward_patients = compute_hts_emergence_ward.compute(org_unit, start_period, end_period)
    emergence_ward_patients_disaggregated = emergenc_ward_patients_disaggregation.compute(emergence_ward_patients, end_period)

    inpatients = compute_hts_inpatient.compute(org_unit, start_period, end_period)
    inpatients_disaggregated = inpatient_patients_disaggregation.compute(inpatients, end_period)

    maltnutrition_patients = compute_hts_malnutrition.compute(org_unit, start_period, end_period)
    malnutrition_patients_disaggregated = malnutrition_patients_disaggregation.compute(maltnutrition_patients, end_period)

    index_case_patients = compute_hts_index_case.compute(org_unit, start_period, end_period)
    index_case_patients_disaggregated = index_case_patients_disaggregation.compute(index_case_patients, end_period)

    pmtct_anc_patients = compute_hts_pmtct_anc.compute(org_unit, start_period, end_period)
    pmtct_anc_patients_disaggregated = pmtct_anc_patients_disaggregation.compute(pmtct_anc_patients, end_period)

    tb_clinic_patients = compute_hts_tb_clinic.compute(org_unit, start_period, end_period)
    tb_clinic_patients_diaggregated = tb_clinic_patients_disaggregattion.compute(tb_clinic_patients, end_period)

    combination = vct_patients_disaggregated + pregnant_patients_disaggregated + breastfeeding_patients_disaggregated + other_pitc_patients_disaggregated + emergence_ward_patients_disaggregated + inpatients_disaggregated + malnutrition_patients_disaggregated + index_case_patients_disaggregated + pmtct_anc_patients_disaggregated + tb_clinic_patients_diaggregated

    if combination:
        hts = pd.json_normalize(combination)
        hts['period'] = period
        hts = hts[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
        hts_data = pd.read_csv('HTS_DATA.csv')
        hts_data = pd.concat([hts_data, hts])
        hts_data = hts_data.sort_values(['orgUnit', 'dataElement', 'period'])
        hts_data.to_csv('HTS_DATA.csv', index=False)